# Summarization
Lexrank with tfidf & LSH


In [1]:
# %pip install emoji

In [1]:
import os
import sys

nlp_path = os.path.abspath('../')
if nlp_path not in sys.path:
    sys.path.insert(0, nlp_path)
from utils.tweet_preprocessing import tokenizeRawTweetText

In [2]:
from numpy.linalg import norm
from fast_pagerank import pagerank
from sklearn.feature_extraction.text import TfidfVectorizer
from itertools import combinations
from sklearn.metrics.pairwise import cosine_similarity
# from sentence_transformers import SentenceTransformer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import numpy as np
from utils.lsh import LSH
from utils.fast_lexrank import Lexrank
import pandas as pd
import scipy
import pickle
import time
import string
import re
import emoji, string
import nltk
nltk.download('stopwords')

/home/ehoang/miniconda3/envs/py37/lib/python3.7/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0
[nltk_data] Downloading package stopwords to /home/ehoang/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [68]:
folder = "/home/ehoang/hnt/tweet_interpret_sum/datasets/unlabeled_data/2015_nepal_earthquake_en/prediction_data/"
date = "20150427"
output = "/home/ehoang/hnt/tweet_interpret_sum/datasets/sum_groundtruth/2015_nepal_earthquake/"+date+"/"
# folder = "/home/ehoang/hnt/tweet_interpret_sum/datasets/unlabeled_data/2015_nepal_earthquake_en/prediction_data"
# date = "20150425"

In [69]:
def extract_files(folder, date, is_return_files = False, delimiter = "\t"):
    if is_return_files ==True:
        data = []
    else:
        data = pd.DataFrame()
    if os.path.isfile(folder):
        files = [folder]
    else:
        for dir, _, filenames in os.walk(folder):
            while len(filenames) > 0:
                file = filenames.pop()
                if file.endswith(".csv") and date in str(file):
                    if is_return_files ==True:
                        data.append(os.path.join(dir, file))
                    else:
                        temp = pd.read_csv(os.path.join(dir, file), delimiter=delimiter)
                        data = pd.concat([data, temp])
    
    return data

In [70]:
# read all tweets of the days
data_all = extract_files(folder, date, is_return_files = False)
data_all.shape

(352648, 6)

In [71]:
data_all = data_all[['tweet_id', 'tweet_text', 'predicted_label']]
data_all.shape

(352648, 3)

In [72]:
# remove duplicates
print("Data before removing duplicates", data_all.shape)
data_all.drop_duplicates(subset=['tweet_text'], inplace=True, keep='first')
print("Data after removing duplicates", data_all.shape)

Data before removing duplicates (352648, 3)
Data after removing duplicates (155828, 3)


In [73]:
#preprocess text
punctuation = "!\"$%&'()*+,-./:;<=>?@[\]^_`{|}~”…’”—→"
data_all['tweet_text'] = data_all['tweet_text'].apply(lambda x: str(re.sub("^'|'$", "", x)))

data_all['prepro_text'] = data_all['tweet_text'].apply(lambda x: tokenizeRawTweetText(x))
data_all['prepro_text'] = data_all['prepro_text'].apply(lambda x: x.replace("\\n", " "))
lemmatizer  = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))
data_all['prepro_text'] = data_all['prepro_text'].apply(lambda x: ' '.join([lemmatizer.lemmatize(y) for y in x.split(" ")]))
data_all['prepro_text'] = data_all['prepro_text'].apply(lambda x: ' '.join([y for y in x.split(" ") if y not in stop_words]))
data_all['prepro_text'] = data_all['prepro_text'].apply(lambda x: x.translate(str.maketrans('', '', punctuation)).strip())
data_all['prepro_text'] = data_all['prepro_text'].apply(lambda x: ''.join(c for c in x if c not in emoji.UNICODE_EMOJI).strip())
data_all['prepro_text'] = data_all['prepro_text'].apply(lambda x: re.sub(" +", " ", x))


In [74]:
data_all.head(n=2)

,tweet_id,tweet_text,predicted_label,prepro_text
0,592763070298066944,RT @SpokespersonMoD: #NepalEarthquake Casualty...,affected_people_and_evacuations,#nepalearthquake casualty evacuation fine exam...
1,592763069945765888,RT @Harry_Styles: To help those affected by th...,not_related_or_irrelevant,help affected devastating earthquake nepal tex...


In [75]:
# remove duplicates
print("Data before removing duplicates", data_all.shape)
data_all.drop_duplicates(subset=['prepro_text'], inplace=True, keep='first')
print("Data after removing duplicates", data_all.shape)

Data before removing duplicates (155828, 4)
Data after removing duplicates (102918, 4)


In [76]:
data_all = data_all.reset_index(drop=True)

In [77]:
data_all.shape

(102918, 4)

In [78]:
# extract data of the categories
for cat in set(data_all['predicted_label']):
    print("%-40s%20d"%(cat, data_all[data_all['predicted_label'] == cat].shape[0]))

rescue_volunteering_and_donation_effort                22977
other_useful_information                               12689
not_related_or_irrelevant                              42385
infrastructure_and_utilities_damage                     3959
injured_or_dead_people                                 10288
affected_people_and_evacuations                        10620


In [115]:
current_cat = "rescue_volunteering_and_donation_effort"
data = data_all[data_all['predicted_label'] == current_cat]
print(data.shape)
data = data.reset_index(drop=True)

(22977, 4)


## 1. Lexrank + lsh + tfidf

In [116]:
#extract tfidf vector
tfidf = TfidfVectorizer()
tfidfData = tfidf.fit_transform(data['prepro_text'])
print(tfidfData.shape)

(22977, 16702)


In [117]:
lsh_tfidf = LSH(tfidfData)
lsh_tfidf.train(num_bits = 6)


(22977,)


In [118]:
lsh_tfidf.model['bin_indices']

{0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63}

In [ ]:
%%time
lex_tfidf = Lexrank(tfidfData, lsh_tfidf)
lex_tfidf.build_graph(search_radius = 3, cosine_sim = 0.3)

#buckets: 64
.......Buck: 0, vec: (15831, 16702)


In [ ]:
lex_tfidf.train(lexrank_iter = 100, damping_factor = 0.85)

In [ ]:
sentIds = lex_tfidf.extract_summary(n_sents = 200, cosine_thres=0.5)

In [ ]:
print("Id", "#adjacentEdges", "lexrank")
for i, idx in enumerate(sentIds):
    print(i, len(lex_tfidf.graph[idx]), lex_tfidf.scores[idx])

In [ ]:
# nearby_bin = 0
with open(output+current_cat+".txt", "w") as f:
    for i, idx in enumerate(sentIds):
        print("{}.\t{}\t{}".format(i, data.iloc[idx]['tweet_id'], data.iloc[idx]['tweet_text']))
        f.write("{}.\t{}\t{}\n".format(i, data.iloc[idx]['tweet_id'], data.iloc[idx]['tweet_text']))

In [29]:
# # nearby_bin = 0
# for i, idx in enumerate(sentIds):
#     print(i, data.iloc[idx]['tweet_id'], data.iloc[idx]['prepro_text'])